# Задание 6

### Пример входных данных

In [1]:
import pandas as pd
clinton = pd.read_csv("../clinton.csv", encoding="utf-8")
trump = pd.read_csv("../trump.csv", encoding="utf-8")

In [2]:
trump.head()

,text
0,Join me for a 3pm rally - tomorrow at the Mid-...
1,"Once again, we will have a government of, by a..."
2,"On National #VoterRegistrationDay, make sure y..."
3,Hillary Clinton's Campaign Continues To Make F...
4,"'CNBC, Time magazine online polls say Donald T..."


In [3]:
clinton.head()

,text
0,The question in this election: Who can put the...
1,"Last night, Donald Trump said not paying taxes..."
2,Couldn't be more proud of @HillaryClinton. Her...
3,"If we stand together, there's nothing we can't..."
4,Both candidates were asked about how they'd co...


# Задание 6

## Описание задания

Предлагается написать генератор случайных диалогов на основе твитов Трампа (файл <i>"trump.csv"</i>) и Хеллори Клинтон (файл <i>"clinton.csv"</i>).

* Каждый диалог состоит из нескольких "ходов" (<i>turn</i>).
* Каждый такой <i>turn</i> представляет собой цепочку нескольких реплик разных агентов (участников диалога, их может быть больше двух).
* Цепочка реплик представляет собой набор реплик агентов. Первое сообщение в цепочке - <i>"исходное сообщение"</i> на которое отвечают оставшиеся агенты (по одному сообщению за turn).

Программа должна представлять отдельный пакет следующей структуры:
<ul>
<li>run\_generator.py</li>
<li>dialogsgenerator:
<ul>
<li>agent.py</li>
<li>randomdialog.py</li>
<li>\_\_init__.py</li>
</ul></li>
</ul>

<b>URGENT:</b> в работе ЗАПРЕЩАЕТСЯ использовать циклы:
<ul>
<li>Использование 1 for (или while): max 1 балл</li>
<li>Использование до 4 for (или while): max 0.8 баллов</li>
<li>Использование 4+ for (или while): max 0.5 баллов</li>
</ul>

### Файл agent.py

Описание класса агента (один класс для всех агентов, в нашем случае для Трампа и Клинтон).

In [4]:
'''
from collections import Generator
import random

class Agent(Generator):
    
    def __init__(self, kb, name):
        self.name = name
        self.kb = kb

    def send(self, msg):
            
        ###
        ### Generate answer...
        ###

        return random.choice(self.kb.values)[0]

    def throw(self, typ=None, val=None, tb=None):
        # Оставляем как есть
        super().throw(typ, val, tb)

    def __str__(self):
        head = 5
        return "Name {0}".format(self.name) + "answers:" + "\n".join(list(*zip(*self.kb.values[:head])))

    def __repr__(self):
        return 'Agent({0})'.format(self.name)
'''

'\nfrom collections import Generator\nimport random\n\nclass Agent(Generator):\n    \n    def __init__(self, kb, name):\n        self.name = name\n        self.kb = kb\n\n    def send(self, msg):\n            \n        ###\n        ### Generate answer...\n        ###\n\n        return random.choice(self.kb.values)[0]\n\n    def throw(self, typ=None, val=None, tb=None):\n        # Оставляем как есть\n        super().throw(typ, val, tb)\n\n    def __str__(self):\n        head = 5\n        return "Name {0}".format(self.name) + "answers:" + "\n".join(list(*zip(*self.kb.values[:head])))\n\n    def __repr__(self):\n        return \'Agent({0})\'.format(self.name)\n'

#### Пример использования:

In [5]:
from dialogsgenerator.agent import Agent

#Как "увидеть" Agent в agent, который уже в package?

Agent(clinton, "clinton"), Agent(trump, "trump")

(Agent(clinton), Agent(trump))

### Файл randomdialog.py

Описание класса, генерирующего случайный диалог.

In [6]:
'''
import sys

class RandomDialog(object):
    
    def __init__(self, agents, max_len=10):
        self.agents = agents
        self.max_len = max_len

    def generate(self):
        """
        Генерирует случайный диалог состоящий из 1-max_len цепочек.
        При генерации вызывает функцию turn.
        Возвращаемый объект является генератором.
        """
        
        return (yield from list(map(lambda x: self.turn(), range(self.max_len))))
        
    def turn(self):
        """
        Генерирует одну случайную цепочку следующим образом: выбирается случайный агент.
        Он "говорит" случайное сообщение (msg) из своей Agent.kb (используйте next или send(None)).
        (правила того, как выбирать никак не регулируются, вы можете выбирать случайный твит из Agent.kb никак не учитывая
        переданное msg).
        Это сообщение передается с помощью send (помним, что агент - это объект-генератор).
        Далее получаем ответ от всех агентов и возвращаем (генерируем) их (включая исходное сообщение).
        Возвращаемый объект является генератором.
        """
        
        #Почему возвращаемый объект -- генератор, если ответы уже получены? Что генерировать?
        
        agent = random.choice(self.agents)
        agents = self.agents.copy()
        agents.remove(agent)
        firstMsg = agent.send(None)
        
        return [agent.name + " : " + firstMsg] + list(map(lambda x: agent.name + " : " + agent.send(firstMsg), agents))

    def eval(self, dialog=None):
        """
        Превращает генератор случайного далога (который возвращается в self.generate())
        в список списков (пример использования далее).
        Возвращаемый объект является списком.
        """
        if dialog is None:
            dialog = self.generate()
        
        return list(map(lambda x: next(dialog), range(self.max_len)))
            
    def write(self, dialog=None, target=sys.stdout):
        """
        Записывает результат self.eval() в соответствующий поток.
        """
        if dialog is None:
            dialog = self.eval()
        
        def printInFile(msgs):
            target.write('turn {0}\n'.format(msgs[1]))
            target.write("\n".join(msgs[0]))
            target.write('\n\n')
        
        list(map(printInFile, zip(dialog, range(len(dialog)))))
'''

'\nimport sys\n\nclass RandomDialog(object):\n    \n    def __init__(self, agents, max_len=10):\n        self.agents = agents\n        self.max_len = max_len\n\n    def generate(self):\n        """\n        Генерирует случайный диалог состоящий из 1-max_len цепочек.\n        При генерации вызывает функцию turn.\n        Возвращаемый объект является генератором.\n        """\n        \n        return (yield from list(map(lambda x: self.turn(), range(self.max_len))))\n        \n    def turn(self):\n        """\n        Генерирует одну случайную цепочку следующим образом: выбирается случайный агент.\n        Он "говорит" случайное сообщение (msg) из своей Agent.kb (используйте next или send(None)).\n        (правила того, как выбирать никак не регулируются, вы можете выбирать случайный твит из Agent.kb никак не учитывая\n        переданное msg).\n        Это сообщение передается с помощью send (помним, что агент - это объект-генератор).\n        Далее получаем ответ от всех агентов и возв

#### Пример использования:

In [7]:
from dialogsgenerator.randomdialog import RandomDialog
rd = RandomDialog([Agent(clinton, "clinton"), Agent(trump, "trump")], max_len=5)

In [8]:
generated = rd.generate()
generated

<generator object RandomDialog.generate at 0x1053a4ca8>

In [9]:
evaled = rd.eval(generated)
evaled

[['trump : #Trump2016 #IACaucus Finder:\nhttps://t.co/ANvTcZ8EpQ https://t.co/gicaoEOFCC',
  'trump : Will be on @FallonTonight with @JimmyFallon on @NBC at 11:35pmE. Enjoy! #Trump2016 https://t.co/Z9zbKfXJqb'],
 ['clinton : So this is kind of scary... https://t.co/8t63MRC9x6 https://t.co/qaCWNBd0bq',
  'clinton : Too many moms have to go back to work too soon after giving birth.\nhttps://t.co/aYxAdmuSAJ'],
 ['trump : Readout of my meeting with Israeli Prime Minister Benjamin Netanyahu:\nhttps://t.co/VHuyE65hgi https://t.co/6KDPrQxSED',
  'trump : I agree with Marco Rubio that Ted Cruz is a liar!'],
 ['trump : New York, we will make America great again! https://t.co/KyxbaW4fsL',
  'trump : "@gene70: @FireFlyFury @megynkelly @realDonaldTrump And this is the bimbo that\'s asking presidential questions? https://t.co/oU1uUGnuWb"'],
 ['trump : Under the leadership of Obama &amp; Clinton, Americans have experienced more attacks at home than victories abroad. Time to change the playbook!',
  

In [10]:
rd.write(evaled)

turn 0
trump : #Trump2016 #IACaucus Finder:
https://t.co/ANvTcZ8EpQ https://t.co/gicaoEOFCC
trump : Will be on @FallonTonight with @JimmyFallon on @NBC at 11:35pmE. Enjoy! #Trump2016 https://t.co/Z9zbKfXJqb

turn 1
clinton : So this is kind of scary... https://t.co/8t63MRC9x6 https://t.co/qaCWNBd0bq
clinton : Too many moms have to go back to work too soon after giving birth.
https://t.co/aYxAdmuSAJ

turn 2
trump : Readout of my meeting with Israeli Prime Minister Benjamin Netanyahu:
https://t.co/VHuyE65hgi https://t.co/6KDPrQxSED
trump : I agree with Marco Rubio that Ted Cruz is a liar!

turn 3
trump : New York, we will make America great again! https://t.co/KyxbaW4fsL
trump : "@gene70: @FireFlyFury @megynkelly @realDonaldTrump And this is the bimbo that's asking presidential questions? https://t.co/oU1uUGnuWb"

turn 4
trump : Under the leadership of Obama &amp; Clinton, Americans have experienced more attacks at home than victories abroad. Time to change the playbook!
trump : Crooked 

### Файл run_generator.py

Содержит функции для генерации и записи нескольких диалогов. Файл должен быть написан так, чтобы его можно было запускать через командную строку:
<img src="../cmd.png">
Описание аргументов представлено на рисунке:
<img src="../cmd_help.png">
Для разбора аргументов ипользуйте <a href="https://docs.python.org/3/howto/argparse.html">модуль argparse</a>. Все представленные на рисунке аргументы должны быть обработаны (кроме help, он обрабатывается автоматически модулем argparse).

In [14]:
#См. сразу файл.

Hello World!


usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/volodden/Library/Jupyter/runtime/kernel-e1f5822b-263e-4afe-a228-bbc181480c96.json


SystemExit: 2

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### Пример использования функций из файла:

In [ ]:
import sys
from run_generator import generate, write

In [ ]:
dialogs = generate(rd, 2)
dialogs

In [ ]:
write(dialogs, sys.stdout)